<a href="https://colab.research.google.com/github/JananiSudeeptha/Stroke-Prediction/blob/hyperOpt/XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv("/content/drive/Shareddrives/Data Mining Project/Datasets/Kaggle/stroke_data.csv")

In [4]:
df['sex'] = df['sex'].fillna(df['sex'].mode()[0])

In [5]:
df = df.drop(df.index[df['age'] < 0])

In [6]:
df = df.drop(df.index[df['age'] >=100])

In [7]:
df = df[['hypertension', 'avg_glucose_level', 'heart_disease', 'age', 'bmi', 'work_type', 'stroke']]

In [8]:
y = df['stroke']
X = df.drop(['stroke'], axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from xgboost import XGBClassifier
params = {
            'objective':'binary:logistic',
            'max_depth': 4,
            'alpha': 10,
            'learning_rate': 1.0,
            'n_estimators':100
        }         
           
          
# instantiate the classifier 
xgb_clf = XGBClassifier(**params)


# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1.0, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, ...)

In [13]:
y_pred = xgb_clf.predict(X_test)
print(f"training score : {xgb_clf.score(X_train, y_train)}")
print(f"test score : {xgb_clf.score(X_test, y_test)}")
print(classification_report(y_test, y_pred))

training score : 0.9930187697112588
test score : 0.9897121861604409
              precision    recall  f1-score   support

           0       1.00      0.98      0.99      4090
           1       0.98      1.00      0.99      4075

    accuracy                           0.99      8165
   macro avg       0.99      0.99      0.99      8165
weighted avg       0.99      0.99      0.99      8165



In [28]:
y_pred = cross_val_predict(xgb_clf, X, y, cv=10)

In [29]:
report = classification_report(y, y_pred)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99     20450
           1       0.99      1.00      0.99     20374

    accuracy                           0.99     40824
   macro avg       0.99      0.99      0.99     40824
weighted avg       0.99      0.99      0.99     40824



In [31]:
from sklearn.model_selection import GridSearchCV

# Initialize classifier
xgb_model = XGBClassifier()

param_grid = {
    "learning_rate": [0.05, 0.1, 0.15],
    "max_depth": [3, 5, 7],
    "n_estimators": [100, 500, 1000],
}
# Perform grid search cross-validation
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring="accuracy",
)
grid_search.fit(X, y)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.15],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [100, 500, 1000]},
             scoring='accuracy')

In [32]:
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

Best Parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 1000}
Best Score: 1.0


In [ ]:
y_pred = cross_val_predict(grid_search, X, y, cv=5)
report = classification_report(y, y_pred)
print(report)